In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,clone_model
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

import keras.backend as K

Using TensorFlow backend.


In [3]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [4]:
from utils import *
df = functions.parse_file(r"raw_data/EmoContext/train3.txt", "EmoContext")
df.head(5)

,id,label,turn1,turn2,turn3
0,0,others,dont worry ism girl,hmm how do i know if you are,whats ur name ?
1,1,angry,when did in,saw many times i think,no . i never saw you
2,2,others,by,by google chrome,where you live
3,3,angry,u r ridiculous,i might be ridiculous but i am telling the tru...,u little disgusting whore
4,4,others,just for time pass,wt do u do a a living then,maybe


In [5]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [6]:
NR_WORDS = 5000
MAX_PROP_LENGTH = 100
tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

tokenizer.fit_on_texts(text_data)

X_train = tokenizer.texts_to_sequences(text_data)
X_train = pad_sequences(X_train, maxlen = MAX_PROP_LENGTH)

In [7]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_train_others = []
Y_train_angry = []
Y_train_sad = []
Y_train_happy = []

for idx,row in df.iterrows():
    Y_train_others.append(one_hot_vector(row['label'],"others"))
    Y_train_angry.append(one_hot_vector(row['label'],"angry"))
    Y_train_sad.append(one_hot_vector(row['label'],"sad"))
    Y_train_happy.append(one_hot_vector(row['label'],"happy"))

Y_train_others = np.array(Y_train_others)
Y_train_angry = np.array(Y_train_angry)
Y_train_sad = np.array(Y_train_sad)
Y_train_happy = np.array(Y_train_happy)

In [8]:
df = functions.parse_file(r"raw_data/EmoContext/devwithlabels.txt", "EmoContext")
df.head(5)

,id,label,turn1,turn2,turn3
0,0,angry,Then dont ask me,YOURE A GUY NOT AS IF YOU WOULD UNDERSTAND,IM NOT A GUY FUCK OFF
1,1,others,Mixed things such as??,the things you do.,Have you seen minions??
2,2,happy,Today I'm very happy,and I'm happy for you ❤,I will be marry
3,3,others,Woah bring me some,left it there oops,Brb
4,4,others,it is thooooo,I said soon master.,he is pressuring me


In [9]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [10]:
#tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
#                                   lower=True,split=' ')

#tokenizer.fit_on_texts(text_data)

X_test = tokenizer.texts_to_sequences(text_data)
X_test = pad_sequences(X_test, maxlen = MAX_PROP_LENGTH)

In [11]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_test_others = []
Y_test_angry = []
Y_test_sad = []
Y_test_happy = []

for idx,row in df.iterrows():
    Y_test_others.append(one_hot_vector(row['label'],"others"))
    Y_test_angry.append(one_hot_vector(row['label'],"angry"))
    Y_test_sad.append(one_hot_vector(row['label'],"sad"))
    Y_test_happy.append(one_hot_vector(row['label'],"happy"))

Y_test_others = np.array(Y_test_others)
Y_test_angry = np.array(Y_test_angry)
Y_test_sad = np.array(Y_test_sad)
Y_test_happy = np.array(Y_test_happy)

In [12]:
embed_dim = 64
lstm_out = 32
batch_size = 128

adam = optimizers.Adam(lr=0.01)
rmsprop = optimizers.RMSprop(lr=0.01, rho=0.95, epsilon=None, decay=0.0)


model = Sequential()
model.add(Embedding(NR_WORDS, embed_dim,input_length = X_train.shape[1],name="boosting_embeding"))
model.add(LSTM(lstm_out))
model.add(Dense(2,activation='softmax'))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
boosting_embeding (Embedding (None, 100, 64)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 332,482
Trainable params: 332,482
Non-trainable params: 0
_________________________________________________________________


In [14]:
mdcheck = ModelCheckpoint("trained_models/best_model_val_acc{val_acc:.4f}.h5", monitor='val_f1', save_best_only=True)

In [15]:
Y_train_angry = np.array(Y_train_angry)

model_angry = clone_model(model)
model_angry.compile(loss = 'binary_crossentropy', optimizer="adam", metrics = ['accuracy', f1])
model_angry.name = "angry_model"
history = model_angry.fit(X_train, Y_train_angry,
                    validation_data=(X_test, Y_test_angry),
                    epochs=5, verbose=1, batch_size=batch_size,callbacks=[mdcheck],shuffle=True,
                    class_weight={
                        0: len(X_train) / len(np.where(Y_train_angry[:,0]==1.0)[0]),
                        1: len(X_train) / len(np.where(Y_train_angry[:,1]==1.0)[0]),
                    })

Train on 30160 samples, validate on 2755 samples
Epoch 1/5


InternalError: failed initializing StreamExecutor for CUDA device ordinal 0: Internal: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 8505131008

In [ ]:
save_model(model_angry,indices=[{"indices":["turn1","turn2","turn3"],"NR_WORDS":NR_WORDS,"MAX_PROP_LENGTH":MAX_PROP_LENGTH}])

In [ ]:
Y_train_sad = np.array(Y_train_sad)

model_sad = clone_model(model)
model_sad.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])
model_sad.name = "sad_model"
history = model_sad.fit(X_train, Y_train_sad,
                    validation_data=(X_test, Y_test_sad),
                    epochs=5, verbose=1, batch_size=batch_size,callbacks=[mdcheck],
                    class_weight={
                        0: len(X_train) / len(np.where(Y_train_sad[:,0]==1.0)[0]),
                        1: len(X_train) / len(np.where(Y_train_sad[:,1]==1.0)[0]),
                    })

In [ ]:
save_model(model_sad,indices=[{"indices":["turn1","turn2","turn3"],"NR_WORDS":NR_WORDS,"MAX_PROP_LENGTH":MAX_PROP_LENGTH}])

In [ ]:
Y_train_happy = np.array(Y_train_happy)

model_happy = clone_model(model)
model_happy.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])
model_happy.name = "happy_model"
history = model_happy.fit(X_train, Y_train_happy,
                    validation_data=(X_test, Y_test_happy),
                    epochs=5, verbose=1, batch_size=batch_size,callbacks=[mdcheck],
                    class_weight={
                        0: len(X_train) / len(np.where(Y_train_happy[:,0]==1.0)[0]),
                        1: len(X_train) / len(np.where(Y_train_happy[:,1]==1.0)[0]),
                    })

In [ ]:
save_model(model_happy,indices=[{"indices":["turn1","turn2","turn3"],"NR_WORDS":NR_WORDS,"MAX_PROP_LENGTH":MAX_PROP_LENGTH}])

In [ ]:
Y_train_others = np.array(Y_train_others)

model_others = clone_model(model)
model_others.compile(loss = 'binary_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])
model_others.name = "others_model"
history = model_others.fit(X_train, Y_train_others,
                    validation_data=(X_test, Y_test_others),
                    epochs=5, verbose=1, batch_size=batch_size,callbacks=[mdcheck],
                    class_weight={
                        0: len(X_train) / len(np.where(Y_train_others[:,0]==1.0)[0]),
                        1: len(X_train) / len(np.where(Y_train_others[:,1]==1.0)[0]),
                    })

In [ ]:
save_model(model_others,indices=[{"indices":["turn1","turn2","turn3"],"NR_WORDS":NR_WORDS,"MAX_PROP_LENGTH":MAX_PROP_LENGTH}])